*model implement, use human data on cc subontology, other model is all the same except for the dimension of labels.*

**使用之前的脚本获取数据来训练我们的多源数据的模型集成**
**数据有蛋白质序列数据（已表示为向量）、ppi网络数据（已表示为向量，128维）**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
import pickle
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_score, recall_score, f1_score, average_precision_score
import numpy as np
import math

In [2]:
Thresholds = list(map(lambda x:round(x*0.01,2), list(range(1,100))))

In [3]:
with open('cc_ic.pkl','rb')as f:
    ic = pickle.load(f)

In [4]:
##数据集
#import numpy as np
class MyDataset(Dataset):
    def __init__(self,embpath='bp_data/emb_train.pkl',ppipath='bp_data/ppi256.train',labelpath='bp_data/label_train.pkl'):
        super().__init__()
        with open(embpath,'rb')as f:
            self.emb=torch.tensor(pickle.load(f)).permute(0,2,1).cuda()
            
        with open(ppipath,'rb')as f:
            self.ppi=torch.tensor(pickle.load(f)).cuda()
        with open(labelpath,'rb')as f:
            self.label=torch.tensor(pickle.load(f)).float().cuda()
        
    def __getitem__(self,index):
        return self.emb[index],self.ppi[index],self.label[index]
    def __len__(self):
        return self.emb.shape[0]

In [5]:
class GONET(nn.Module):
    def __init__(self, func='CC',batch_size=32,output_size=298,hidden_size=128,dropout=0.3):
        super(GONET, self).__init__()
        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.dropout = dropout
        
        self.cnn = self.SeqConv1d().cuda()     
        self.rnn = nn.GRU(750, self.hidden_size,num_layers=3,batch_first=True,dropout=self.dropout,bidirectional=True).cuda()
        
        self.seq_FClayer = nn.Linear(2*30*self.hidden_size+128, 2000).cuda()
        self.label=nn.Linear(2000, self.output_size).cuda()
        self.W_s1 = nn.Linear(2*self.hidden_size, 350).cuda()
        self.W_s2 = nn.Linear(350, 30).cuda()
        
#     def attention_net(self, lstm_output, final_state):
#         if len(final_state.shape)>2:
#             hidden = final_state.squeeze(0)
#         else:
#             hidden=final_state
#         attn_weights = torch.bmm(lstm_output, hidden.unsqueeze(2)).squeeze(2)
#         soft_attn_weights = F.softmax(attn_weights, 1)
#         new_hidden_state = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)

#         return new_hidden_state
    def self_attn(self, lstm_output):
        attn_weight_matrix = self.W_s2(torch.tanh(self.W_s1(lstm_output)))
        attn_weight_matrix = attn_weight_matrix.permute(0, 2, 1)
        attn_weight_matrix = F.softmax(attn_weight_matrix, dim=2)

        return attn_weight_matrix

    def forward(self, seqMatrix,ppiVec):
        
        seq_out = self.cnn(seqMatrix)
        output, _ = self.rnn(seq_out) 
        attn_weight_matrix = self.self_attn(output)
        
        attention_value = torch.bmm(attn_weight_matrix, output)
        
        attention_value = attention_value.view(-1, attention_value.size()[1]*attention_value.size()[2])
        
        final = torch.cat((attention_value, ppiVec),1)
        final = F.dropout(self.seq_FClayer(final), p=self.dropout, training=self.training)
        final = F.relu(final)
        final = self.label(final)
        return torch.sigmoid(final)


    # sequence的1D_CNN模型
    def SeqConv1d(self):
        layers = []
        in_channels = 100
        layers += [nn.Conv1d(in_channels, 512, kernel_size=16, stride=1, padding=8),
                           nn.ReLU(inplace=True)]
        layers += [nn.MaxPool1d(kernel_size=2)]
        layers += [nn.Conv1d(512, 128, kernel_size=16, stride=1, padding=8),
                           nn.ReLU(inplace=True)]
        layers += [nn.MaxPool1d(kernel_size=2, stride=1)]
        layers += [nn.Conv1d(128, 32, kernel_size=16, stride=1, padding=8),
                           nn.ReLU(inplace=True)]
        layers += [nn.MaxPool1d(kernel_size=2, stride=1)]       
        return nn.Sequential(*layers)

In [6]:
def cacul_aupr(lables, pred):
    precision, recall, _thresholds = metrics.precision_recall_curve(lables, pred)
    aupr = metrics.auc(recall, precision)
    return aupr

In [7]:
def calculate_performance(actual, pred_prob, threshold=0.4, average='micro'):
    pred_lable = []
    for l in range(len(pred_prob)):
        eachline = (np.array(pred_prob[l]) > threshold).astype(np.int)
        eachline = eachline.tolist()
        pred_lable.append(eachline)
    f_score = f1_score(np.array(actual), np.array(pred_lable), average=average)
    recall = recall_score(np.array(actual), np.array(pred_lable), average=average)
    precision = precision_score(np.array(actual), np.array(pred_lable), average=average)
    return f_score, precision, recall

In [8]:
with open('cc_ic1.pkl','rb')as f:
    ic = pickle.load(f)
ic = torch.tensor(ic).cuda()
def eval_s(real_annots, pred_annots,t):
    total = 0
    ru = 0.0
    mi = 0.0

    for i in range(len(real_annots)):
        if len(real_annots[i]) == 0:
            continue
        #eachline = (np.array(pred_annots[i]) > threshold).astype(np.int).tolist()
        pred = torch.tensor(pred_annots[i]).cuda()
        real= torch.tensor(real_annots[i]).cuda()
        pred = (pred>=t).float()
        tp = pred*real
    
        fp = pred-tp
        #print(sum(tp)/sum(fp))
        
        fn = real-tp
        mi+=sum(ic*fp)
        ru+=sum(ic*fn)
        #print(mi,ru)
        total += 1
       
    ru /= total
    mi /= total
   
    s = math.sqrt(ru * ru + mi * mi)
    return s

In [9]:
def train_and_test(learningrate=0.001,batchsize=32,epochtime=6,func='cc'):
    print('{}  seqmodel start'.format(func))
    seq_model = GONET(func).cuda()
    batch_size = batchsize
    learning_rate = learningrate
    epoch_times = epochtime
    print(seq_model)
    print('batch_size_{},learning_rate_{},epoch_times_{}'.format(batch_size, learning_rate, epoch_times))
    loss_function = nn.BCELoss()
    optimizer = optim.Adam(seq_model.parameters(), lr=learning_rate,weight_decay=0.00001)

    train_dataset = MyDataset(embpath='homo/cc/emb_train.pkl',ppipath='homo/cc/ppi_train.pkl',labelpath='homo/cc/labels_train.pkl')
    train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataset = MyDataset(embpath='homo/cc/emb_test.pkl',ppipath='homo/cc/ppi_test.pkl',labelpath='homo/cc/labels_test.pkl')
    test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

    seq_model.train()
    best_fscore = 0
    
    for epoch in range(epoch_times):
        _loss = 0
        batch_num = 0
        for batch_idx, (seqMatrix, ppiVect, labels) in enumerate(train_data_loader):
            seqMatrix = Variable(seqMatrix).cuda()
            ppiVect = Variable(ppiVect).cuda()
            #GO_annotiations = torch.squeeze(GO_annotiations)
            GO_annotiations = Variable(labels).cuda()
            out = seq_model(seqMatrix,ppiVect)
            optimizer.zero_grad()
            loss = loss_function(out, GO_annotiations)
            batch_num += 1
            loss.backward()
            optimizer.step()
            _loss += loss.item()
#             if batch_idx %20 == 0:
#                 print("epoch={},batch={},loss={:.6f}".format(epoch,batch_idx,loss.item()))
        epoch_loss = "{}".format(_loss / batch_num)
        t_loss = 0
        test_batch_num = 0
        pred = []
        actual = []
        for idx, (seqMatrix, ppiVect, GO_annotiations) in enumerate(test_data_loader):
            seqMatrix = Variable(seqMatrix).cuda()
            ppiVect = Variable(ppiVect).cuda()
            GO_annotiations = Variable(GO_annotiations).cuda()
            out = seq_model(seqMatrix,ppiVect)
            test_batch_num = test_batch_num + 1
            pred.append(out.data[0].cpu().tolist())
            actual.append(GO_annotiations.data[0].cpu().tolist())
            one_loss = loss_function(out, GO_annotiations)
            t_loss += one_loss.item()
        test_loss = "{}".format(t_loss / test_batch_num)
        fpr, tpr, th = roc_curve(np.array(actual).flatten(), np.array(pred).flatten(), pos_label=1)
        auc_score = auc(fpr, tpr)
        aupr=cacul_aupr(np.array(actual).flatten(), np.array(pred).flatten())
        score_dict = {}
        each_best_fcore = 0

        each_best_scores = []
        smin=1000
        for i in range(len(Thresholds)):
            f_score,precision, recall  = calculate_performance(actual, pred, threshold=Thresholds[i])
            if f_score >= each_best_fcore:
                each_best_fcore = f_score
                each_best_scores = [Thresholds[i], f_score, recall, precision, auc_score]
            scores = [f_score, recall, precision, auc_score]
            score_dict[Thresholds[i]] = scores
            #s = evaluate_annotations(actual,pred,Thresholds[i])
            #smin = min(smin,s)
        if each_best_fcore >= best_fscore:
            best_fscore = each_best_fcore
            best_scores = each_best_scores
            best_score_dict = score_dict
            torch.save(seq_model, 'savedpkl/Seq1DNVal_{}_{}_{}_{}.pkl'.format(func, batch_size, learning_rate, epoch_times))
        t, f_score, recall = each_best_scores[0], each_best_scores[1], each_best_scores[2]
        precision, auc_score = each_best_scores[3], each_best_scores[4]
        print('epoch{},loss{},testloss:{},t:{},f_score{}, auc{}, recall{}, precision{},aupr{}'.format(
            epoch, epoch_loss, test_loss, t, f_score, auc_score, recall, precision,aupr))
    bestthreshold, f_max, recall_max = best_scores[0], best_scores[1], best_scores[2]
    prec_max, bestauc_score = best_scores[3], best_scores[4]
    print('lr:{},batch:{},epoch{},f_max:{}\nauc{},recall_max{},prec_max{},threshold:{}'.format(
        learning_rate, batch_size, epoch_times,
        f_max, bestauc_score, recall_max, prec_max, bestthreshold,smin))
    test_Seqmodel = torch.load('savedpkl/Seq1DNVal_{}_{}_{}_{}.pkl'.format(func, batch_size, learning_rate, epoch_times)).cuda()
    t_loss = 0
    seq_test_outs = {}
    # seq_test_outs = []
    pred = []
    actual = []
    score_dict = {}
    batch_num = 0
    for batch_idx, (seqMatrix, ppiVect, GO_annotiations) in enumerate(test_data_loader):
        seqMatrix = Variable(seqMatrix).cuda()
        ppiVect = Variable(ppiVect).cuda()
        GO_annotiations = Variable(GO_annotiations).cuda()
        out = test_Seqmodel(seqMatrix,ppiVect)
        batch_num += 1
        seq_test_outs[batch_idx-1] = out.data[0].cpu().tolist()
        pred.append(out.data[0].cpu().tolist())
        actual.append(GO_annotiations.data[0].cpu().tolist())
        loss = loss_function(out, GO_annotiations)
        t_loss += loss.item()
    test_loss = "{}".format(t_loss / batch_num)
    fpr, tpr, th = roc_curve(np.array(actual).flatten(), np.array(pred).flatten(), pos_label=1)
    auc_score = auc(fpr, tpr)
    aupr=cacul_aupr(np.array(actual).flatten(), np.array(pred).flatten())
    each_best_fcore = 0
    for i in range(len(Thresholds)):
        f_score, precision,recall  = calculate_performance(actual, pred, threshold=Thresholds[i])
        if f_score > each_best_fcore:
            each_best_fcore = f_score
            each_best_scores = [Thresholds[i], f_score, recall, precision, auc_score]
        scores = [f_score, recall, precision, auc_score]
        score_dict[Thresholds[i]] = scores
#         smin = min(smin,evaluate_annotations(actual,pred,Thresholds[i]))
    bestthreshold, f_max, recall_max = each_best_scores[0], each_best_scores[1], each_best_scores[2]
    prec_max, bestauc_score = each_best_scores[3], each_best_scores[4]

    print('test_loss:{},lr:{},batch:{},epoch{},f_max:{}\nauc_score{},recall_max{},prec_max{},threshold:{},aupr{}'.format(
        test_loss, learning_rate, batch_size, epoch_times,
        f_max, auc_score ,recall_max, prec_max, bestthreshold,aupr))
    return pred,actual
#     with open('out/weight_out/Seqout{}_lr{}_bat{}_epo{}.csv'.format(
#             func, learning_rate, batch_size, epoch_times), 'w') as f:
#         f.write('lr:{},batchsize:{},epochtimes:{}\n'.format(learning_rate, batch_size, epoch_times))
#         f.write('f_max:{},recall_max{},prec_max{},auc_score:{}\n'.format(
#             f_max,recall_max, prec_max, auc_score))
#         f.write('threshold,f_score,recall,precision, roc_auc,auc\n')
#         for i in range(len(Thresholds)):
#             f.write('{},'.format(str(Thresholds[i])))
#             f.write('{}\n'.format(','.join(str(x) for x in score_dict[Thresholds[i]])))
#         for key, var in seq_test_outs.items():
#             f.write('{},'.format(str(key)))
#             f.write('{}\n'.format(','.join(str(x) for x in var)))
#         #获取再最优模型下的训练集的输出
#     train_out_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=False)
#     seq_train_outs = {}
#          #返回再最优的Seq模型下的训练集的输出和测试集的输出，用于训练weight_classifier
#     for batch_idx, (seqMatrix, ppiVect, GO_annotiations) in enumerate(train_out_loader):
#         seqMatrix = Variable(seqMatrix).cuda()
#         ppiVect = Variable(ppiVect).cuda()
#         # GO_annotiations = Variable(GO_annotiations).cuda()
#         out = test_Seqmodel(seqMatrix,ppiVect)
#         seq_train_outs[batch_idx] = out.data[0].cpu().tolist()
#     return (seq_train_outs, seq_test_outs,bestthreshold) 


In [10]:
pred,actual= train_and_test()

cc  seqmodel start
GONET(
  (cnn): Sequential(
    (0): Conv1d(100, 512, kernel_size=(16,), stride=(1,), padding=(8,))
    (1): ReLU(inplace=True)
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(512, 128, kernel_size=(16,), stride=(1,), padding=(8,))
    (4): ReLU(inplace=True)
    (5): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(128, 32, kernel_size=(16,), stride=(1,), padding=(8,))
    (7): ReLU(inplace=True)
    (8): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (rnn): GRU(750, 128, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
  (seq_FClayer): Linear(in_features=7808, out_features=2000, bias=True)
  (label): Linear(in_features=2000, out_features=298, bias=True)
  (W_s1): Linear(in_features=256, out_features=350, bias=True)
  (W_s2): Linear(in_features=350, out_features=30, bias=True)
)
batch_size_32,learning_rate_0.001,epoch_times_

/home/wanglx/enter/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type GONET. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


epoch0,loss0.1380339864828543,testloss:0.10516875619160089,t:0.33,f_score0.6615143253947842, auc0.9513982748910395, recall0.6352511144864718, precision0.6900427906477619,aupr0.7258786770335497
epoch1,loss0.1111964413632541,testloss:0.0987116303166181,t:0.33,f_score0.6793118045858634, auc0.9591834484285087, recall0.6638387730152262, precision0.6955233536033122,aupr0.7493411060326535
epoch2,loss0.10597489449327395,testloss:0.0973858415545473,t:0.35,f_score0.6823201821226318, auc0.9603709225040195, recall0.6717667543437188, precision0.6932104900986314,aupr0.7552188323113026
epoch3,loss0.10295519780486687,testloss:0.09605819806211591,t:0.36,f_score0.6852407739071211, auc0.9617527782721229, recall0.6709146815841144, precision0.7001920276313585,aupr0.760418250738373
epoch4,loss0.10053325279458472,testloss:0.09540903784656034,t:0.39,f_score0.6889397504916025, auc0.9622406690526948, recall0.6597883401869621, precision0.7207862288535737,aupr0.763613470566301
epoch5,loss0.09821665628928028,testl

In [11]:
s = eval_s(actual,pred,.45)

In [12]:
s

11.240694054186521

In [13]:

def eval_s(real_annots, pred_annots,t):
    total = 0
    ru = 0.0
    mi = 0.0

    for i in range(len(real_annots)):
        if len(real_annots[i]) == 0:
            continue
        #eachline = (np.array(pred_annots[i]) > threshold).astype(np.int).tolist()
        pred = torch.tensor(pred_annots[i]).cuda()
        real= torch.tensor(real_annots[i]).cuda()
        pred = (pred>=t).float()
        tp = pred*real
    
        fp = pred-tp
        #print(sum(tp)/sum(fp))
        
        fn = real-tp
        print(sum(fp))
        print(sum(tp))
        print(sum(fn))
        mi+=sum(ic*fp)
        ru+=sum(ic*fn)
        #print(mi,ru)
        total += 1
       
    ru /= total
    mi /= total
   
    s = math.sqrt(ru * ru + mi * mi)
    return s

In [16]:
s1  = eval_s(actual,pred,.42)

tensor(0., device='cuda:0')
tensor(17., device='cuda:0')
tensor(6., device='cuda:0')
tensor(5., device='cuda:0')
tensor(5., device='cuda:0')
tensor(2., device='cuda:0')
tensor(4., device='cuda:0')
tensor(5., device='cuda:0')
tensor(2., device='cuda:0')
tensor(0., device='cuda:0')
tensor(7., device='cuda:0')
tensor(11., device='cuda:0')
tensor(0., device='cuda:0')
tensor(7., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(7., device='cuda:0')
tensor(0., device='cuda:0')
tensor(5., device='cuda:0')
tensor(5., device='cuda:0')
tensor(2., device='cuda:0')
tensor(0., device='cuda:0')
tensor(7., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(20., device='cuda:0')
tensor(5., device='cuda:0')
tensor(0., device='cuda:0')
tensor(7., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(7., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(19., device='cuda:0')
tensor(17., devi

tensor(4., device='cuda:0')
tensor(26., device='cuda:0')
tensor(7., device='cuda:0')
tensor(5., device='cuda:0')
tensor(8., device='cuda:0')
tensor(1., device='cuda:0')
tensor(0., device='cuda:0')
tensor(18., device='cuda:0')
tensor(11., device='cuda:0')
tensor(1., device='cuda:0')
tensor(9., device='cuda:0')
tensor(3., device='cuda:0')
tensor(0., device='cuda:0')
tensor(20., device='cuda:0')
tensor(7., device='cuda:0')
tensor(3., device='cuda:0')
tensor(15., device='cuda:0')
tensor(33., device='cuda:0')
tensor(6., device='cuda:0')
tensor(5., device='cuda:0')
tensor(0., device='cuda:0')
tensor(4., device='cuda:0')
tensor(5., device='cuda:0')
tensor(7., device='cuda:0')
tensor(1., device='cuda:0')
tensor(11., device='cuda:0')
tensor(10., device='cuda:0')
tensor(2., device='cuda:0')
tensor(14., device='cuda:0')
tensor(3., device='cuda:0')
tensor(10., device='cuda:0')
tensor(7., device='cuda:0')
tensor(5., device='cuda:0')
tensor(6., device='cuda:0')
tensor(5., device='cuda:0')
tensor(10.

tensor(8., device='cuda:0')
tensor(3., device='cuda:0')
tensor(2., device='cuda:0')
tensor(0., device='cuda:0')
tensor(16., device='cuda:0')
tensor(9., device='cuda:0')
tensor(1., device='cuda:0')
tensor(9., device='cuda:0')
tensor(40., device='cuda:0')
tensor(3., device='cuda:0')
tensor(22., device='cuda:0')
tensor(20., device='cuda:0')
tensor(2., device='cuda:0')
tensor(8., device='cuda:0')
tensor(31., device='cuda:0')
tensor(6., device='cuda:0')
tensor(9., device='cuda:0')
tensor(8., device='cuda:0')
tensor(4., device='cuda:0')
tensor(9., device='cuda:0')
tensor(6., device='cuda:0')
tensor(1., device='cuda:0')
tensor(16., device='cuda:0')
tensor(0., device='cuda:0')
tensor(1., device='cuda:0')
tensor(8., device='cuda:0')
tensor(29., device='cuda:0')
tensor(11., device='cuda:0')
tensor(4., device='cuda:0')
tensor(0., device='cuda:0')
tensor(11., device='cuda:0')
tensor(2., device='cuda:0')
tensor(1., device='cuda:0')
tensor(12., device='cuda:0')
tensor(7., device='cuda:0')
tensor(0.,

tensor(11., device='cuda:0')
tensor(5., device='cuda:0')
tensor(0., device='cuda:0')
tensor(2., device='cuda:0')
tensor(11., device='cuda:0')
tensor(27., device='cuda:0')
tensor(3., device='cuda:0')
tensor(29., device='cuda:0')
tensor(7., device='cuda:0')
tensor(9., device='cuda:0')
tensor(21., device='cuda:0')
tensor(5., device='cuda:0')
tensor(0., device='cuda:0')
tensor(10., device='cuda:0')
tensor(52., device='cuda:0')
tensor(5., device='cuda:0')
tensor(7., device='cuda:0')
tensor(0., device='cuda:0')
tensor(6., device='cuda:0')
tensor(14., device='cuda:0')
tensor(4., device='cuda:0')
tensor(16., device='cuda:0')
tensor(5., device='cuda:0')
tensor(0., device='cuda:0')
tensor(9., device='cuda:0')
tensor(3., device='cuda:0')
tensor(2., device='cuda:0')
tensor(0., device='cuda:0')
tensor(13., device='cuda:0')
tensor(17., device='cuda:0')
tensor(0., device='cuda:0')
tensor(7., device='cuda:0')
tensor(3., device='cuda:0')
tensor(9., device='cuda:0')
tensor(9., device='cuda:0')
tensor(1.

tensor(8., device='cuda:0')
tensor(9., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(8., device='cuda:0')
tensor(0., device='cuda:0')
tensor(4., device='cuda:0')
tensor(8., device='cuda:0')
tensor(0., device='cuda:0')
tensor(6., device='cuda:0')
tensor(5., device='cuda:0')
tensor(2., device='cuda:0')
tensor(2., device='cuda:0')
tensor(5., device='cuda:0')
tensor(3., device='cuda:0')
tensor(4., device='cuda:0')
tensor(8., device='cuda:0')
tensor(0., device='cuda:0')
tensor(1., device='cuda:0')
tensor(8., device='cuda:0')
tensor(0., device='cuda:0')
tensor(7., device='cuda:0')
tensor(13., device='cuda:0')
tensor(5., device='cuda:0')
tensor(0., device='cuda:0')
tensor(7., device='cuda:0')
tensor(1., device='cuda:0')
tensor(3., device='cuda:0')
tensor(8., device='cuda:0')
tensor(0., device='cuda:0')
tensor(7., device='cuda:0')
tensor(8., device='cuda:0')
tensor(0., device='cuda:0')
tensor(6., device='cuda:0')
tensor(8., device='cuda:0')
tensor(0., device='

...

tensor(4., device='cuda:0')
tensor(5., device='cuda:0')
tensor(3., device='cuda:0')
tensor(0., device='cuda:0')
tensor(18., device='cuda:0')
tensor(1., device='cuda:0')
tensor(0., device='cuda:0')
tensor(18., device='cuda:0')
tensor(2., device='cuda:0')
tensor(7., device='cuda:0')
tensor(5., device='cuda:0')
tensor(3., device='cuda:0')
tensor(3., device='cuda:0')
tensor(7., device='cuda:0')
tensor(15., device='cuda:0')
tensor(1., device='cuda:0')
tensor(14., device='cuda:0')
tensor(20., device='cuda:0')
tensor(1., device='cuda:0')
tensor(8., device='cuda:0')
tensor(24., device='cuda:0')
tensor(0., device='cuda:0')
tensor(26., device='cuda:0')
tensor(27., device='cuda:0')
tensor(0., device='cuda:0')
tensor(7., device='cuda:0')
tensor(7., device='cuda:0')
tensor(0., device='cuda:0')
tensor(21., device='cuda:0')
tensor(4., device='cuda:0')
tensor(9., device='cuda:0')
tensor(9., device='cuda:0')
tensor(0., device='cuda:0')
tensor(2., device='cuda:0')
tensor(16., device='cuda:0')
tensor(11.

tensor(1., device='cuda:0')
tensor(18., device='cuda:0')
tensor(1., device='cuda:0')
tensor(5., device='cuda:0')
tensor(6., device='cuda:0')
tensor(8., device='cuda:0')
tensor(0., device='cuda:0')
tensor(17., device='cuda:0')
tensor(6., device='cuda:0')
tensor(2., device='cuda:0')
tensor(17., device='cuda:0')
tensor(5., device='cuda:0')
tensor(0., device='cuda:0')
tensor(19., device='cuda:0')
tensor(6., device='cuda:0')
tensor(1., device='cuda:0')
tensor(30., device='cuda:0')
tensor(18., device='cuda:0')
tensor(9., device='cuda:0')
tensor(5., device='cuda:0')
tensor(1., device='cuda:0')
tensor(11., device='cuda:0')
tensor(16., device='cuda:0')
tensor(10., device='cuda:0')
tensor(3., device='cuda:0')
tensor(12., device='cuda:0')
tensor(5., device='cuda:0')
tensor(8., device='cuda:0')
tensor(5., device='cuda:0')
tensor(9., device='cuda:0')
tensor(4., device='cuda:0')
tensor(9., device='cuda:0')
tensor(2., device='cuda:0')
tensor(3., device='cuda:0')
tensor(25., device='cuda:0')
tensor(3.

tensor(1., device='cuda:0')
tensor(15., device='cuda:0')
tensor(1., device='cuda:0')
tensor(1., device='cuda:0')
tensor(12., device='cuda:0')
tensor(1., device='cuda:0')
tensor(6., device='cuda:0')
tensor(7., device='cuda:0')
tensor(7., device='cuda:0')
tensor(5., device='cuda:0')
tensor(5., device='cuda:0')
tensor(2., device='cuda:0')
tensor(0., device='cuda:0')
tensor(18., device='cuda:0')
tensor(14., device='cuda:0')
tensor(1., device='cuda:0')
tensor(21., device='cuda:0')
tensor(11., device='cuda:0')
tensor(5., device='cuda:0')
tensor(35., device='cuda:0')
tensor(8., device='cuda:0')
tensor(1., device='cuda:0')
tensor(23., device='cuda:0')
tensor(16., device='cuda:0')
tensor(1., device='cuda:0')
tensor(17., device='cuda:0')
tensor(1., device='cuda:0')
tensor(7., device='cuda:0')
tensor(17., device='cuda:0')
tensor(0., device='cuda:0')
tensor(1., device='cuda:0')
tensor(18., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(18., device='cuda:0')
tensor(

tensor(0., device='cuda:0')
tensor(13., device='cuda:0')
tensor(4., device='cuda:0')
tensor(12., device='cuda:0')
tensor(3., device='cuda:0')
tensor(2., device='cuda:0')
tensor(0., device='cuda:0')
tensor(5., device='cuda:0')
tensor(2., device='cuda:0')
tensor(8., device='cuda:0')
tensor(5., device='cuda:0')
tensor(0., device='cuda:0')
tensor(8., device='cuda:0')
tensor(3., device='cuda:0')
tensor(2., device='cuda:0')
tensor(7., device='cuda:0')
tensor(5., device='cuda:0')
tensor(2., device='cuda:0')
tensor(1., device='cuda:0')
tensor(17., device='cuda:0')
tensor(21., device='cuda:0')
tensor(7., device='cuda:0')
tensor(15., device='cuda:0')
tensor(5., device='cuda:0')
tensor(6., device='cuda:0')
tensor(11., device='cuda:0')
tensor(8., device='cuda:0')
tensor(17., device='cuda:0')
tensor(5., device='cuda:0')
tensor(0., device='cuda:0')
tensor(6., device='cuda:0')
tensor(7., device='cuda:0')
tensor(6., device='cuda:0')
tensor(0., device='cuda:0')
tensor(35., device='cuda:0')
tensor(15., 

tensor(0., device='cuda:0')
tensor(13., device='cuda:0')
tensor(4., device='cuda:0')
tensor(3., device='cuda:0')
tensor(15., device='cuda:0')
tensor(13., device='cuda:0')
tensor(3., device='cuda:0')
tensor(17., device='cuda:0')
tensor(1., device='cuda:0')
tensor(2., device='cuda:0')
tensor(18., device='cuda:0')
tensor(5., device='cuda:0')
tensor(16., device='cuda:0')
tensor(6., device='cuda:0')
tensor(0., device='cuda:0')
tensor(7., device='cuda:0')
tensor(14., device='cuda:0')
tensor(3., device='cuda:0')
tensor(8., device='cuda:0')
tensor(3., device='cuda:0')
tensor(2., device='cuda:0')
tensor(3., device='cuda:0')
tensor(6., device='cuda:0')
tensor(7., device='cuda:0')
tensor(1., device='cuda:0')
tensor(11., device='cuda:0')
tensor(10., device='cuda:0')
tensor(10., device='cuda:0')
tensor(21., device='cuda:0')
tensor(4., device='cuda:0')
tensor(2., device='cuda:0')
tensor(5., device='cuda:0')
tensor(0., device='cuda:0')
tensor(12., device='cuda:0')
tensor(4., device='cuda:0')
tensor(0

In [17]:
s1

11.175358667954326